In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.regularizers import l2
import tensorflow_datasets as tfds
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Завантаження даних Fashion MNIST за допомогою TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Нормалізація даних до діапазону [0, 1]
def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_train.map(normalize_img)
ds_test = ds_test.map(normalize_img)

# Функція для побудови моделі з заданими гіперпараметрами
def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(units=hp.Int('units_1', min_value=64, max_value=512, step=32),
                    activation=hp.Choice('dense_activation_1', values=['relu', 'tanh', 'sigmoid']),
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    
    # Додавання додаткових шарів аналогічним чином
    # Додайте шар Dense, BatchNormalization, Dropout для кожного додаткового шару
    
    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss=SparseCategoricalCrossentropy(),
                  metrics=[SparseCategoricalAccuracy()])
    
    return model

# Ініціалізація тюнера
tuner = RandomSearch(
    build_model,
    objective='val_sparse_categorical_accuracy',
    max_trials=10,  # Кількість спроб для підбору
    directory='keras_tuner',  # Каталог для зберігання результатів
    project_name='fashion_mnist_hyperparameter_tuning'
)

# Пошук оптимальних гіперпараметрів
tuner.search(ds_train.batch(128), epochs=30, validation_data=ds_test.batch(128), verbose=2)

# Отримання оптимальних гіперпараметрів
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Побудова моделі з оптимальними гіперпараметрами
best_model = tuner.hypermodel.build(best_hps)

# Навчання моделі з оптимальними гіперпараметрами
best_model.fit(ds_train.batch(128), epochs=30, validation_data=ds_test.batch(128), verbose=2)

# Оцінка точності на тестовому наборі
test_loss, test_acc = best_model.evaluate(ds_test.batch(128), verbose=2)
print(f"Test accuracy: {test_acc}")


ModuleNotFoundError: No module named 'resource'